In [1]:
from datetime import date, timedelta
import io
import os

import requests
from minio import Minio

In [2]:
client = Minio(
        endpoint="192.168.86.192:9000",
        access_key="minio",
        secret_key="minio123",
        secure=False,
    )

In [3]:
if not client.bucket_exists("landing-rws"):
    client.make_bucket("landing-rws")
else:
    print('Bucket landing-rws already exists')

Bucket landing-rws already exists


In [5]:
response = requests.get('https://downloads.rijkswaterstaatdata.nl/filedata/2021-01_rws_filedata.csv')

In [18]:
response.status_code

200

In [16]:
type(response.content)

bytes

In [13]:
import pandas as pd

In [30]:
rawData = pd.read_csv(io.StringIO(response.content.decode('utf-8')), sep=';')

In [32]:
rawData.head(10)

,NLSitNummer,DatumFileBegin,DatumFileEind,TijdFileBegin,TijdFileEind,FileZwaarte,GemLengte,FileDuur,HectometerKop,HectometerStaart,...,TrajVan,TrajNaar,OorzaakGronddetail,OorzaakVerloop,OorzaakCodeVerloop,OorzaakCode,Oorzaak_1,Oorzaak_2,Oorzaak_3,Oorzaak_4
0,2800676,2021-01-27,2021-01-27,16:40:00,16:42:56,"8,800","3000,000","2,933","54,8","51,8",...,Amersfoort,Zwolle,Spitsfile (geen oorzaak gemeld),[Geen oorzaakcode opgegeven door VWM 3],[000],001,Spitsfile (geen oorzaak gemeld),Geen oorzaak gemeld,Drukte,Hoge intensiteit
1,2800645,2021-01-27,2021-01-27,15:40:00,16:48:59,"348,067","5045,663","68,983","41,1","46,1",...,Breda,Rotterdam,Spoedreparatie,"[Geen oorzaakcode opgegeven door VWM 1], [Spoe...","[000], [N32], [CLE], [000]",N32,Spoedreparatie,Spoedreparatie,Spoedreparatie,Ongepland onderhoud
2,2800680,2021-01-27,2021-01-27,16:58:00,17:45:53,"250,767","5237,035","47,883","88,4","92,4",...,Nijmegen,Maasbracht,Defecte vrachtwagen(s),"[Geen oorzaakcode opgegeven door VWM 4], [Defe...","[000], [HBD], [M13]",HBD,Defecte vrachtwagen(s),Defecte vrachtwagen,Incident (gestrand voertuig),Incident
3,2800633,2021-01-27,2021-01-27,14:17:00,14:37:00,"40,000","2000,000","20,000","25,9","23,9",...,Arnhem,Arnhem Velperbroek,File buiten spits (geen oorzaak gemeld),[Geen oorzaakcode opgegeven door VWM 20],[000],003,File buiten spits (geen oorzaak gemeld),Geen oorzaak gemeld,Drukte,Hoge intensiteit
4,2800686,2021-01-27,2021-01-27,17:51:00,18:02:59,"23,967","2000,000","11,983","22,5","28,5",...,Zwolle,Amersfoort,Ongeval(len),"[Geen oorzaakcode opgegeven door VWM 1], [Onge...","[000], [ACI], [ACA]",ACI,Ongeval(len),Ongeval,Ongeval,Ongeval
5,2800729,2021-01-28,2021-01-28,7:55:00,8:30:59,"97,000","2695,693","35,983","136,3","138,3",...,Oberhausen,Arnhem,Spitsfile (geen oorzaak gemeld),[Geen oorzaakcode opgegeven door VWM 36],[000],001,Spitsfile (geen oorzaak gemeld),Geen oorzaak gemeld,Drukte,Hoge intensiteit
6,2800852,2021-01-29,2021-01-29,8:01:00,8:45:16,"186,433","4211,596","44,267","143,2","148,2",...,Oss,'s-Hertogenbosch,Obstakel(s) op de weg {iets dat de weg of een ...,[Obstakel(s) op de weg {iets dat de weg of een...,"[OHX], [SPL], [000]",OHX,Obstakel(s) op de weg {iets dat de weg of een ...,Lading op de weg,Incident (obstakel op de weg),Incident
7,2800810,2021-01-28,2021-01-28,18:02:00,18:18:58,"40,967","2414,538","16,967","32,9","34,9",...,Gouda,Hoek van Holland,Spitsfile (geen oorzaak gemeld),[Geen oorzaakcode opgegeven door VWM 17],[000],001,Spitsfile (geen oorzaak gemeld),Geen oorzaak gemeld,Drukte,Hoge intensiteit
8,2800770,2021-01-28,2021-01-28,15:40:00,15:56:59,"36,933","2174,681","16,983","23,1","20,1",...,Rotterdam,Breda,Ongeval(len),"[Geen oorzaakcode opgegeven door VWM 2], [Onge...","[000], [ACI], [ACA]",ACI,Ongeval(len),Ongeval,Ongeval,Ongeval
9,2800786,2021-01-28,2021-01-28,16:27:00,18:25:09,"438,417","3710,679","118,150","51,2","53,2",...,Ridderkerk,Europoort,Spitsfile (met ongeval op aansluitende weg),"[Geen oorzaakcode opgegeven door VWM 60], [Ong...","[000], [N65], [000], [M14], [000]",001,Spitsfile (geen oorzaak gemeld),Geen oorzaak gemeld,Drukte,Hoge intensiteit


In [4]:
def save_file_to_landing_rws(name):
    result = client.put_object(
        "landing-rws", name, io.BytesIO(response.content), -1,
        content_type="application/csv", part_size=10*1024*1024,
        )
    print(
        "created {0} object; etag: {1}, version-id: {2}".format(
        result.object_name, result.etag, result.version_id,
        ),
    )

In [7]:
endpoint = 'https://downloads.rijkswaterstaatdata.nl/filedata/'
file = f'{year}-{month}_rws_filedata.csv'

In [5]:
def fetch_and_save(endpoint, name):
    response = requests.get(endpoint+name)
    result = client.put_object(
        "landing-rws", name, io.BytesIO(response.content), -1,
        content_type="application/csv", part_size=10*1024*1024,
        )
    print(
        "created {0} object; etag: {1}, version-id: {2}".format(
        result.object_name, result.etag, result.version_id,
        ),
    )

In [42]:
[range(12)]

[range(0, 12)]

In [43]:
range(2015, 2022)

range(2015, 2022)

In [9]:
for year in range(2022, 2023):
    for month in range(1,13):
        fetch_and_save(endpoint, f'{year}-{month:02}_rws_filedata.csv')

created 2022-01_rws_filedata.csv object; etag: 94c287154dc4f55234fd03a23888708a, version-id: None
created 2022-02_rws_filedata.csv object; etag: 7fb08f89675b2add3d4ce7b7eadc2395, version-id: None
created 2022-03_rws_filedata.csv object; etag: e13c51b1bdbf98678463ceb8b604f6f7, version-id: None
created 2022-04_rws_filedata.csv object; etag: 0f19562f15ef1bd07071b93153a4997e, version-id: None
created 2022-05_rws_filedata.csv object; etag: 5e81843583676e0055af80f069f36df6, version-id: None
created 2022-06_rws_filedata.csv object; etag: 13fdfe914c93e7dcbb47592cb11e547d-2, version-id: None
created 2022-07_rws_filedata.csv object; etag: a02bdb664cdaf2a2438d330e207bacde, version-id: None
created 2022-08_rws_filedata.csv object; etag: e5254a5a11bf185e429114fd9240d9e2, version-id: None
created 2022-09_rws_filedata.csv object; etag: 3cc6eca9e0a76424a501c5d2bbaa0927, version-id: None
created 2022-10_rws_filedata.csv object; etag: 4496e0a4f594f857769112ecab59008c, version-id: None
created 2022-11_rw